In [ ]:
# pip install crosslingual-coreference
import spacy#, coreferee
nlp = spacy.load("fr_core_news_lg")
# nlp.add_pipe('coreferee')


In [ ]:
import spacy
nlp = spacy.load('fr_core_news_lg')
nlp.add_pipe(
    "xx_coref", config={"chunk_size": 2500, "chunk_overlap": 2, "device": 0}
)


In [ ]:
import sys
sys.path.append('..')
from extract_corpus import get_corpus

corpus = get_corpus('../data')


In [ ]:

# for year in corpus:
#   print(year)

for year in ["1918"]:
# for year in corpus:
  year_text = "\n".join([chunk.text for chunk in corpus[year]]) #le but c'est de concatener le texte de chaque chunk de 1917
  doc = nlp(year_text)
  persons = [ent for ent in doc.ents if ent.label_ == "PER"]
  print(f"Year {year:<20}")
  for person in persons:
    if (person.root.head.pos_ == "ADJ" and person.root.dep_=="nsubj"):
      print(f"{person.text:<20} {person.root.head.text:<20} {person.root.head.pos_:<10} {person.root.dep_:<10}")
  print("=================")


In [ ]:
year_text = "\n".join([chunk.text for chunk in corpus['1814']]) #le but c'est de concatener le texte de chaque chunk de 1917
doc = nlp(year_text)

In [ ]:
# doc =  nlp("Mme JOUDKIA est jolie. Elle mange des fleurs. Monsieur LEBORGNE est malade. Il est à l'hopital.")

adjectifs = [token for token in doc if token.pos_ == "ADJ"]
for adj in adjectifs:
  for child in adj.children:
      if child.dep_ == "nsubj":
        subj = None
        for chunk in doc.noun_chunks:
          if (chunk[0].i <= child.i) and (child.i <= chunk[-1].i):
            subj = chunk
        if not subj:
          subj = child
        print(f"{adj.text:<20} || {subj.text:<20} || {subj.ents}")
        print("=================")



# persons = [ent for ent in doc.ents if ent.label_ == "PER"]
# for person in persons:
#   if (person.root.head.pos_ == "ADJ" and person.root.dep_=="nsubj"):
#     print(f"{person.text:<20} {person.root.head.text:<20} {person.root.head.pos_:<10} {person.root.dep_}")
# print("=================")

In [ ]:
spacy.displacy.serve(doc, style="dep")#, options={"ents": ["PER"]})

In [ ]:
import stanza

nlp = stanza.Pipeline("fr")

doc = nlp("Mme JOUDKIA est jolie. Elle mange des fleurs. Monsieur LEBORGNE est malade. Il est à l'hopital.")

In [ ]:

def recursive_find_adjs(root, sent):
    children = [w for w in sent.words if w.head == root.id]

    if not children:
        return []

    filtered_c = [w for w in children if w.deprel == "conj" and w.upos == "ADJ"]
    # Do not include an adjective if it is the parent of a noun to prevent
    results = [w for w in filtered_c if not any(sub.head == w.id and sub.upos == "NOUN" for sub in sent.words)]
    for w in children:
        results += recursive_find_adjs(w, sent)

    return results

for sent in doc.sentences:
    nouns = [w for w in sent.words if w.upos == "NOUN"]
    # print(nouns[0].pretty_print())
    # print("==")
    noun_adj_pairs = {}
    for noun in nouns:
        # Find constructions in the form of "La voiture est belle"
        # In this scenario, the adjective is the parent of the noun
        cop_root = sent.words[noun.head-1]
        adjs = [cop_root] + recursive_find_adjs(cop_root, sent) if cop_root.upos == "ADJ" else []

        # Find constructions in the form of "La femme intelligente et belle"
        # Here, the adjectives are descendants of the noun
        mod_adjs = [w for w in sent.words if w.head == noun.id and w.upos == "ADJ"]
        # This should only be one element because conjunctions are hierarchical
        if mod_adjs:
            mod_adj = mod_adjs[0]
            adjs.extend([mod_adj] + recursive_find_adjs(mod_adj, sent))

        if adjs:
            unique_adjs = []
            unique_ids = set()
            for adj in adjs:
                if adj.id not in unique_ids:
                    unique_adjs.append(adj)
                    unique_ids.add(adj.id)

            noun_adj_pairs[noun.text] = " ".join([adj.text for adj in unique_adjs])

    print(noun_adj_pairs)